redis

this notebook has code to insert the raw_html dataset as small chunks that refer to their parents, and a python object to store the parents (original docs). the code also will save the .rdb (if running redis-stack locally through docker) and the pickled parent store object to the current directory.

at the end of the notebook are two code cells that demo how to use all this for retrieval

get_html() or at least part of it I think might be useful because it makes the formatting of wiki pages nice for the recursive chunking

a further idea is having the parent documents be chunked based on section, but I don't think any page is long enough to have to do that.

In [2]:
# from langchain.retrievers import ParentDocumentRetriever
from OurParentDocumentRetriever import OurParentDocumentRetriever
from langchain.storage import InMemoryStore  # ...
from langchain_redis import RedisConfig, RedisVectorStore

# from langchain_community.storage import RedisStore
# from langchain_community.vectorstores.redis import Redis
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from bs4 import BeautifulSoup
from transformers import AutoTokenizer
from langchain_core.documents import Document
from langchain_huggingface import HuggingFaceEmbeddings
import re
import redis
import pickle

In [3]:
def get_html(file, sep=" "):

    # stolen from https://stackoverflow.com/a/75501596 this sucks
    NON_BREAKING_ELEMENTS = [
        "a",
        "abbr",
        "acronym",
        "audio",
        "b",
        "bdi",
        "bdo",
        "big",
        "button",
        "canvas",
        "cite",
        "code",
        "data",
        "datalist",
        "del",
        "dfn",
        "em",
        "embed",
        "i",
        "iframe",
        "img",
        "input",
        "ins",
        "kbd",
        "label",
        "map",
        "mark",
        "meter",
        "noscript",
        "object",
        "output",
        "picture",
        "progress",
        "q",
        "ruby",
        "s",
        "samp",
        "script",
        "select",
        "slot",
        "small",
        "span",
        "strong",
        "sub",
        "sup",
        "svg",
        "template",
        "textarea",
        "time",
        "u",
        "tt",
        "var",
        "video",
        "wbr",
    ]

    def html_to_text(text, preserve_new_lines=True, strip_tags=["style", "script"]):
        soup = BeautifulSoup(text, "html.parser")
        for element in soup(strip_tags):
            element.extract()
        prev = False
        if preserve_new_lines:
            for element in soup.find_all():
                strings = element.find_all(string=True, recursive=False)
                if strings is not None:
                    strings = [
                        s
                        for s in (None if s.strip() == "" else s for s in strings)
                        if s is not None
                    ]
                strings = strings is not None and strings != []
                strings = True
                if element.name not in NON_BREAKING_ELEMENTS and strings:
                    (
                        element.append("\n")
                        if element.name == "br"
                        else element.append("\n\n")
                    )
        return soup.get_text(separator="")  # close enough

    def replace_newlines(text):
        text = re.sub(r"\n{3}", "\n", text)
        text = re.sub(r"\n{4,}", "\n\n", text)
        return text

    with open(file, "r") as f:
        soup = BeautifulSoup(f, "html.parser")
        iwant = soup.find_all("div", {"id": "content"})
        assert len(iwant) == 1
        text = html_to_text(str(iwant[0]))
        # text = soup.get_text(separator=sep)
        # text = str(iwant[0]) # recursivecharactertextsplitter has html splitting built in so i will try that
        # it sucks nevermind
        text = replace_newlines(text).strip()
        return text  # idk how to do better idk why there's just double spaces sometimes
    return "?"

In [4]:
dataset_path = "../dataset/raw_html"
files = os.listdir(dataset_path)
pages = []

for f in files:
    pages.append(get_html(dataset_path + "/" + f))

docs = []
for i, page in enumerate(pages):
    docs.append(Document(page, metadata={"name": files[i]}))

child_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)  # characters not toks

In [5]:
out = child_splitter.split_documents([docs[212]])
print(docs[0].page_content)
for o in out:
    print(o.page_content)
    print("SPLIT-------------------------")

store = (
    InMemoryStore()
)  # theres a way to do this using a different database still in redis but idk how
embedding_model = HuggingFaceEmbeddings(
    model_name="dunzhang/stella_en_1.5B_v5",
    model_kwargs={
        "trust_remote_code": True,
    },
)

config = RedisConfig(
    index_name="umiacs",
    redis_url="redis://localhost:6379",
    metadata_schema=[
        {"name": "doc_id", "type": "tag"},
        {"name": "name", "type": "tag"},
    ],
)
redis_conn = redis.from_url("redis://localhost:6379", db=0)
vectorstore = RedisVectorStore(embedding_model, config)

retriever = OurParentDocumentRetriever(
    vectorstore=vectorstore,
    docstore=store,
    child_splitter=child_splitter,
)

Accounts

From UMIACS

Jump to navigation
Jump to search
This page outlines the various account types at UMIACS and their management methods.
Do not share any UMIACS-affiliated account for any purpose. Action may be taken against shared accounts.

Contents

1 UMIACS Account
2 Guest / Collaborator Account
3 Request a UMIACS account
4 Password Requirements
5 Changing account password

5.1 If you know your current password
5.2 If you do not know your current password


6 Sponsored Accounts
7 All Accounts Related Pages


UMIACS Account
A full UMIACS account consists of the following components:

 Active Directory Account
Provides access to UMIACS-supported computers,  secgroup controlled resources, web services such as GitLab, and the VPN.
 Jira Account
Provides access to the JIRA Request Tracker. Please note that it is possible to set a separate password for this account, at which point it will no longer be synced from your Active Directory Account. Changing your Jira password DOES NOT ch

C:\Users\Andrew\.cache\huggingface\modules\transformers_modules\dunzhang\stella_en_1.5B_v5\d03be74b361d4eb24f42a2fe5bd2e29917df4604\modeling_qwen.py:704: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [6]:
def delete_all_vectors(redis_conn, index_name):
    """Deletes all keys in the given Redis namespace."""
    try:
        # Use SCAN to retrieve all keys that belong to the index namespace
        keys = redis_conn.scan_iter(f"{index_name}:*")
        for key in keys:
            redis_conn.delete(key)  # Delete each key
        print(f"All vectors from index '{index_name}' have been deleted.")
    except Exception as e:
        print(f"Error deleting vectors: {e}")

In [7]:
if len(redis_conn.keys()) != 0:
    print("not empty. deleting everything.")
    delete_all_vectors(redis_conn, "umiacs")
    print(len(redis_conn.keys()))

not empty. deleting everything.
All vectors from index 'umiacs' have been deleted.
0


In [8]:
import torch
print(torch.cuda.memory_allocated() / 1024 / 1024 / 1024)
print(torch.cuda.memory_reserved() / 1024 / 1024 / 1024)
print(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024)
# torch.cuda.empty_cache()

9.262935638427734
9.369140625
9.369140625


In [9]:
# retriever.add_documents(docs)
for i, doc in enumerate(docs):
    retriever.add_documents([doc], ids=[files[i]])
    # print(torch.cuda.memory_reserved() / 1024 / 1024 / 1024)
    if torch.cuda.memory_reserved() / 1024 / 1024 / 1024 > 9.7: # lol
        print(i, doc.metadata["name"])
        print(torch.cuda.memory_reserved() / 1024 / 1024 / 1024)
        print("empty cache")
        torch.cuda.empty_cache()
print(
    f"inserted {len(redis_conn.keys())} chunks and {len(list(store.yield_keys()))} docs"
)
assert len(files) == len(list(store.yield_keys())), f"{len(files)} {len(list(store.yield_keys()))}"

6 Apptainer.html
10.59765625
empty cache
18 BitLocker_PersonalUse.html
10.037109375
empty cache
25 ClassAccounts.html
11.404296875
empty cache
33 Compute_DataLocality.html
9.93359375
empty cache
37 ConferenceRooms_TouchPanel.html
9.80078125
empty cache
46 Data_Storage.html
9.783203125
empty cache
62 Google_Drive_Drive_for_Desktop.html
9.79296875
empty cache
94 Lamsub.sh.o127.html
9.94921875
empty cache
99 LocalDataStorage.html
9.947265625
empty cache
102 MacOSPrinting.html
9.869140625
empty cache
104 MailmanFAQ.html
9.8515625
empty cache
105 MailmanListAdmin.html
9.83984375
empty cache
110 Matlab.html
9.83203125
empty cache
117 Modules.html
10.13671875
empty cache
119 MonthlyMaintenanceWindow.html
10.259765625
empty cache
123 Mpich1sub.sh.o167.html
9.92578125
empty cache
130 Network_Troubleshooting.html
10.6484375
empty cache
139 Network_VPN_Windows.html
9.7578125
empty cache
140 Nexus.html
12.291015625
empty cache
142 Nexus_CBCB.html
10.890625
empty cache
143 Nexus_CLIP.html
9.8535156

In [10]:
sub_docs = vectorstore.similarity_search_with_score("python notebook nexus", return_all=True)
print(sub_docs[0][0].page_content)

jupyter notebook --no-browser --port=8889 --ip=0.0.0.0

This will start running the notebook on port 8889. Note: You must keep this shell window open to be able to connect. If the submission node for the cluster you are using is not accessible via the public internet, you must also be on a machine connected to the UMIACS network or connected to our  VPN in order to access the Jupyter notebook once you start the SSH tunnel, so ensure this is the case before starting the tunnel. Then, on your local machine, run

ssh -N -f -L localhost:8888:<NODENAME>:8889 <USERNAME>@<SUBMISSIONNODE>.umiacs.umd.edu


In [11]:
# assuming you ran redis-stack docker image locally using 
# docker run -d --name redis-stack -p 6379:6379 -p 8001:8001 redis/redis-stack:latest
# otherwise manually copy it yourself idk 
!docker cp redis-stack:/data/dump.rdb ./parentdoc.rdb
with open("parentdoc.pkl", "wb") as out:
    pickle.dump(store, out, -1)

In [12]:
query = "how do i run a python notebook in nexus?"
# query = "nexus python notebook"
res = retriever.similarity_search_with_score(query, k=4)
print([(x[0].metadata["name"], x[1]) for x in res])

embedding = embedding_model.client.encode(query, prompt_name="s2p_query")
res = retriever.similarity_search_with_score_by_vector(embedding, k=4)
print("s2p embedding:", [(x[0].metadata["name"], x[1]) for x in res], sep="\n")
# print(*[x[0].page_content for x in res], sep="\n\n\n\n")
# print(len(thing.similarity_search("nexus", k=4)))
# print(len(thing.similarity_search_with_score("nexus", k=4, distance_threshold=.5)))

[('SLURM.html', [0.39104783535, 0.403365552425, 0.406363248825, 0.437749147415])]
s2p embedding:
[('SLURM.html', [0.340046286583, 0.347794413567, 0.348588466644, 0.382137238979])]


usage example

In [2]:
%reset
import gc 
import torch
gc.collect()
torch.cuda.empty_cache()

Nothing done.


In [1]:
from langchain_redis import RedisConfig, RedisVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from OurParentDocumentRetriever import OurParentDocumentRetriever
import pickle

store = None
with open("parentdoc.pkl", "rb") as f: # you need the file to be there
    store = pickle.load(f)
embedding_model = HuggingFaceEmbeddings(
    model_name="dunzhang/stella_en_1.5B_v5",
    model_kwargs={
        "trust_remote_code": True,
    },
)
config = RedisConfig(
    index_name="umiacs",
    redis_url="redis://localhost:6379",
    metadata_schema=[
        {"name": "doc_id", "type": "tag"},
        {"name": "name", "type": "tag"},
    ],
)
vectorstore = RedisVectorStore(embedding_model, config) # you need redis to be there


retriever = OurParentDocumentRetriever( # you need vectorstore and docstore to match i think probably idk
    vectorstore=vectorstore,
    docstore=store,
    # child_splitter=child_splitter, # dont need for retrieval
)

c:\Users\Andrew\miniforge3\lib\site-packages\langchain\retrievers\document_compressors\chain_extract.py:15: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain.chains.llm import LLMChain
c:\Users\Andrew\miniforge3\lib\site-packages\pydantic\_internal\_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
  warnings.warn(message, UserWarning)
c:\Users\Andrew\miniforge3\lib\site-packages\sentence_transf

01:22:30 redisvl.index.index INFO   Index already exists, not overwriting.


C:\Users\Andrew\.cache\huggingface\modules\transformers_modules\dunzhang\stella_en_1.5B_v5\d03be74b361d4eb24f42a2fe5bd2e29917df4604\modeling_qwen.py:704: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [8]:
#duplicate cell from above

query = "how do i run a python notebook in nexus?"
# query = "nexus python notebook"
res = retriever.similarity_search_with_score(query, k=4)
print([(x[0].metadata["name"], x[1]) for x in res])

embedding = embedding_model.client.encode(query, prompt_name="s2p_query")
res = retriever.similarity_search_with_score_by_vector(embedding, k=4)
print("s2p embedding:", [(x[0].metadata["name"], x[1]) for x in res], sep="\n")
# print(*[x[0].page_content for x in res], sep="\n\n\n\n")
# print(len(thing.similarity_search("nexus", k=4)))
# print(len(thing.similarity_search_with_score("nexus", k=4, distance_threshold=.5)))
context = [x[0].page_content for x in res]
print(f"Context:\n", *context, sep="\n")

[('SLURM.html', [0.39104783535, 0.403365552425, 0.406363248825, 0.437749147415])]
s2p embedding:
[('SLURM.html', [0.340046286583, 0.347794413567, 0.348588466644, 0.382137238979])]
Context:

SLURM

From UMIACS

Jump to navigation
Jump to search
Contents

1 Slurm Workload Manager

1.1 Documentation
1.2 Commands

1.2.1 srun
1.2.2 salloc
1.2.3 sbatch
1.2.4 squeue
1.2.5 scancel
1.2.6 sacct
1.2.7 sstat


1.3 Modules
1.4 Running Jupyter Notebook on a Compute Node

1.4.1 Setting up your Python Virtual Environment
1.4.2 Running Jupyter Notebook

2 Quick Guide to translate PBS/TORQUE to Slurm


Slurm Workload Manager
Slurm is an open-source workload manager designed for Linux clusters of all sizes. It provides three key functions. First, it allocates exclusive or non-exclusive access to resources (computer nodes) to users for some duration of time so they can perform work. Second, it provides a framework for starting, executing, and monitoring work (typically a parallel job) on a set of allocate